<a href="https://colab.research.google.com/github/Gabriele3vi/AIProject/blob/main/PINNs_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [11]:
t1 = torch.arange(12)
t1.reshape(3,2,2)

tensor([[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11]]])

In [14]:
# reshape when we don't know the length of the original tensor
t1.reshape(-1, 3)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

# Creating a basic neural network - IRIS dataset example

---



In [17]:
import torch.nn as nn
import torch.nn.functional as F

In [45]:
# Now let's create a Model class that inherits the nn Module
class Model(nn.Module):
  # Input layer: 4 neurons
  # hidden layer 1
  # hidden layer 2
  # output (3 classes)
  def __init__(self, in_features=4, h1=8, h2=9, outfeatures=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, outfeatures)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x


In [46]:
# Let's create an instance of the model
model = Model()

# Importing Data


In [23]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
url = "https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"
my_df = pd.read_csv(url)


In [25]:
my_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [54]:
my_df['species'] = pd.factorize(my_df['species'])[0]

In [55]:
my_df['species']

,species
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


In [60]:
X = my_df.drop(columns=['species'])
y = my_df['species']

In [34]:
# train test split
from sklearn.model_selection import train_test_split

In [61]:
X = X.values
y = y.values

In [69]:
y.dtype

dtype('int64')

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)


In [71]:
y_train = y_train.to(torch.int64)
y_test = y_test.to(torch.int64)

In [72]:
# To use pytorch, we have to convert everything into a torch.Tensor
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [89]:
model = Model()
# Set the loss
loss = nn.CrossEntropyLoss()

# Choose optimizer, learning rate
opt = torch.optim.Adam(model.parameters(), lr=0.01)


In [90]:
epochs = 100
losses = []
for i in range(epochs):
  y_pred = model.forward(X_train)
  l = loss(y_pred, y_train)
  losses.append(l.detach().numpy())
  opt.zero_grad()
  a = l.backward()
  opt.step()
  if i % 10 == 0:
    print(l)
    print(f"Grad: {a}")



tensor(1.0889, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.9296, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.6561, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.4288, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.3140, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.2301, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.1622, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.1135, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.0831, grad_fn=<NllLossBackward0>)
Grad: None
tensor(0.0650, grad_fn=<NllLossBackward0>)
Grad: None


# PINN example in PyTorch
In this section I try to remake Nico's example with PyTorch

In [92]:
class PINN(nn.Module):
  def __init__(self):
    self.input = nn.Linear(2, 64)
    self.h1 = nn.Linear(64, 126)
    self.h2 = nn.Linear(126, 64)
    self.h3 = nn.Linear(64, 64)
    self.h4 = nn.Linear(64, 32)
    self.out = nn.Linear(32, 1)

  def forward(self, x, t):
    r = torch.stack((x,t))
    out = F.tanh(self.input(r))
    out = F.tanh(self.h1(r))
    out = F.tanh(self.h2(r))
    out = F.tanh(self.h3(r))
    out = F.tanh(self.h4(r))
    out = self.out(r)


In [91]:
t1 = torch.arange(10)
t2 = torch.arange(10)

torch.stack((t1, t2))

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])